
# PREMIER PAS AVEC LE [SQL](https://fr.wikipedia.org/wiki/Structured_Query_Language) 


##  En terminal spécialité NSI


Lycéé Gaston Monnerville                                 Année 2020/21 

-------------------------


Le langage SQL est utilisé pour manipuler des [bases de données](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es). Pour faire simple, on utilise les bases de données pour accéder rapidement à une information dans des données qui font parfois plusieurs milliards de lignes.

* Le tableau dont on se sert est trop grand (comme trier 50000 lignes).
* On souhaite faire des opérations sur deux feuilles Excel (associer les lignes de l'une avec celles de l'autre).

Lorsque le volume de données est important, il est impossible de les voir dans leur ensemble. On peut en voir soit une partie soit une aggrégation. Par exemple, la société qui gère les vélib a ouvert l'accès à ses données. Il est possible de télécharger aussi souvent qu'on veut (toutes les minutes par exemple) un état complet des vélos et places disponibles pour toutes les stations de Paris : c'est une table qui s'enrichit de 1300 lignes toutes les minutes.

----------------------------------------------

## 1) CREATION D'UNE BASE DE DONNEES



Avant de commencer, il convient d’importer le module, comme il est coutume de faire avec Python :

In [1]:
import sqlite3



### 1.a) Connexion

Cela fait, nous pouvons nous connecter à une BDD en utilisant la méthode connect et en lui passant l’emplacement du fichier de stockage en paramètre. Si ce dernier n’existe pas, il est alors créé :



In [2]:
connexion = sqlite3.connect("maBaseDeDonnees.db")  #BDD dans le fichier "maBaseDeDonnees.db"



Comme vous pouvez le voir, nous récupérons un objet retourné par la fonction. Celui-ci est de type Connection et il nous permettra de travailler sur la base.

Par ailleurs, il est aussi possible de stocker la BDD directement dans la RAM en utilisant la chaîne clef ":memory:". Dans ce cas, il n’y aura donc pas de persistance des données après la déconnexion.

Exemple : 
--connexion = sqlite3.connect(":memory:")  #BDD dans la RAM--


Mais… en quoi est-ce utile de stocker des informations dans la RAM puisque celles-ci sont perdues quand on se déconnecte ? :o

C’est une bonne question ! Eh bien, premièrement ce qui est stocké dans la RAM est plus rapide d’accès que ce qu’il y a sur le disque dur. Ainsi, certains utiliseront la RAM de sorte à gagner en performance. Ensuite, les bases temporaires sont aussi très utiles pour effectuer des tests, par exemple des tests unitaires qui sont ainsi reproductibles aisément et n’altèrent pas d’éventuelles BDD persistantes




### 1.b) Deconnexion

Que nous soyons connectés avec la RAM ou non, il ne faut pas oublier de nous déconnecter. Pour cela, il nous suffit de faire appel à la méthode close de notre objet Connection.



In [3]:
connexion.close()  #Déconnexion

### 1.c) Type de champ

Comme nous allons bientôt voir comment exécuter des requêtes, il est important de connaître les types disponibles, avec leur correspondance en Python. Voici ci-dessous, un tableau récapitulatif :


 SQLite     |	Python |
 --------   |---------|
 NULL       |	None   |
 INTEGER    |	int    |
 REAL       |	float  |
 TEXT 	    | str par défaut |
 BLOB  	    | bytes |
Dans le sens inverse, les types Python du tableau seront utilisables avec leur correspondance SQLite. Il est vrai que la liste peut s’avérer restreignante. Heureusement, il est possible d’ajouter nos propres types de données.


## 1) INSERTION DE DONNEES DANS UNE BASE 

nous allons créer une table dans la base contenant un répertoire téléphonique contenant le nom, le prénom, l'adresse et le numéro de téléphone fixe des contacts. Nous verrons comment insérer quelques enregistrements.

In [4]:
baseDeDonnees = sqlite3.connect("maBaseDeDonnees.db")
curseur = baseDeDonnees.cursor()
curseur.execute("CREATE TABLE Contacts (id INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT NOT NULL, prenom TEXT NOT NULL, adresse TEXT NOT NULL, telephoneFixe TEXT)") # Création de la base de données
baseDeDonnees.commit() # On envoie la requête SQL
curseur.execute("INSERT INTO Contacts (nom, prenom, adresse, telephoneFixe) VALUES (?, ?, ?, ?)", ("Dupont", "Paul", "15 rue Louis Pasteur 10000 Troyes", "0325997452")) # On ajoute un enregistrement
baseDeDonnees.commit()
baseDeDonnees.close()

Il est possible d'ajouter un enregistrement depuis un dictionnaire. Dans l'exemple, on ajoute plusieurs enregistrements avec une boucle :

In [5]:
baseDeDonnees = sqlite3.connect("maBaseDeDonnees.db")
curseur = baseDeDonnees.cursor()
personnes = [
	{"nom":"Chabot", "prenom":"Martin", "adresse":"18 rue Général Leclerc 13600 La Ciotat", "telephoneFixe":"0499506373"},
	{"nom":"Delbois", "prenom":"Julie", "adresse":"35 rue du Château 77176 Savigny le Temple", "telephoneFixe":"0199836074"},
	{"nom":"Rivard", "prenom":"Christelle", "adresse":"83 rue de Québec 83400 Hyères", "telephoneFixe":"0499687013"}
]
for contact in personnes:
	curseur.execute("INSERT INTO Contacts (nom, prenom, adresse, telephoneFixe) VALUES (:nom, :prenom, :adresse, :telephoneFixe)", contact) # On ajoute un enregistrement depuis un dictionnaire
baseDeDonnees.commit()
idDernierEnregistrement = curseur.lastrowid # Récupère l'ID de la dernière ligne insérée. 
baseDeDonnees.close()

L'exemple suivant illustre comment modifier des données :


In [6]:
baseDeDonnees = sqlite3.connect("maBaseDeDonnees.db")
curseur = baseDeDonnees.cursor()
curseur.execute("UPDATE Contacts SET telephoneFixe = ? WHERE id = ?", ("0598635076", 2))
baseDeDonnees.commit()
baseDeDonnees.close()

## 3) RECUPERER DES DONNEES


Pour récupérer les données, il est possible de récupérer le premier résultat avec fetchone ou de retourner tous les résultats avec fetchall. Voici un premier exemple utilisant fetchone :

In [7]:
baseDeDonnees = sqlite3.connect("maBaseDeDonnees.db")
curseur = baseDeDonnees.cursor()
curseur.execute("SELECT nom, prenom, telephoneFixe FROM Contacts WHERE id = ?", ("2",))
contact = curseur.fetchone()
print(contact)

('Chabot', 'Martin', '0598635076')


Dans l'exemple ci-dessus, la variable contact contient un tuple avec les valeurs du premier enregistrement retourné par la requête.

Voyons à présent comment récupérer plusieurs enregistrements avec la commande fetchall :

In [8]:
baseDeDonnees = sqlite3.connect("maBaseDeDonnees.db")
curseur = baseDeDonnees.cursor()
curseur.execute("SELECT nom, prenom, telephoneFixe FROM Contacts")
for contact in curseur.fetchall():
    print(contact)

('Dupont', 'Paul', '0325997452')
('Chabot', 'Martin', '0598635076')
('Delbois', 'Julie', '0199836074')
('Rivard', 'Christelle', '0499687013')


Nous n'oublions pas de refermer la base.

In [9]:
 baseDeDonnees.close()
    